# Laboratorio 3 - Data Science
Diego Morales - 21146   
Alejandro Ortega - 

In [532]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import os
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models


In [533]:
print(f"Tensor flow version: {tf.__version__}")

Tensor flow version: 2.10.0


## 1. Preparación de datos

In [534]:
# Cargar el archivo .p
with open('Datos_Rotulos_Trafico/entrenamiento.p', 'rb') as file:
    data = pickle.load(file)

# Imprimir las claves disponibles
print(data.keys())

# Inspeccionar un poco más el contenido
for key in data.keys():
    print(f"Key: {key}, Type: {type(data[key])}, Length: {len(data[key])}")

dict_keys(['coords', 'labels', 'features', 'sizes'])
Key: coords, Type: <class 'numpy.ndarray'>, Length: 34799
Key: labels, Type: <class 'numpy.ndarray'>, Length: 34799
Key: features, Type: <class 'numpy.ndarray'>, Length: 34799
Key: sizes, Type: <class 'numpy.ndarray'>, Length: 34799


In [535]:
def load_data(pickle_file):
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)
    return data['features'], data['labels']

In [536]:
X_train, y_train = load_data('Datos_Rotulos_Trafico/entrenamiento.p')
X_validation, y_validation = load_data('Datos_Rotulos_Trafico/validacion.p')
X_test, y_test = load_data('Datos_Rotulos_Trafico/prueba.p')

In [537]:
X_train = X_train / 255.0
X_validation = X_validation / 255.0
X_test = X_test / 255.0


y_train = np.array(y_train)
y_validation = np.array(y_validation)
y_test = np.array(y_test)

In [538]:
X_train.shape

(34799, 32, 32, 3)

In [539]:
y_train.shape

(34799,)

In [540]:
X_validation.shape

(4410, 32, 32, 3)

In [541]:
y_validation.shape

(4410,)

In [542]:
X_test.shape

(12630, 32, 32, 3)

In [543]:
y_test.shape

(12630,)

## 2. Implementación de la arquitectura Le-Net

## 3. Contrucción de modelo

In [544]:
model = models.Sequential()

In [545]:
# Definir la estructura de capas convolucionales, capas de pooling y capas fully connected
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))


In [546]:
model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_72 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_73 (Conv2D)          (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 6, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_74 (Conv2D)          (None, 4, 4, 64)          36928     
                                                                 
 flatten_29 (Flatten)        (None, 1024)            

In [547]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

## 4. Entrenamiento del modelo

In [548]:
# Mostrar cómo cargar los datos de entrenamiento y validación en lotes.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(len(X_train)).batch(128)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_validation, y_validation))
validation_dataset = validation_dataset.batch(128)

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [549]:
history = model.fit(train_dataset, epochs=5, validation_data=validation_dataset)

Epoch 1/5
272/272 [==============================] - 62s 217ms/step - loss: 1.8181 - accuracy: 0.5155 - val_loss: 0.8762 - val_accuracy: 0.7637
Epoch 2/5
272/272 [==============================] - 55s 201ms/step - loss: 0.3837 - accuracy: 0.8976 - val_loss: 0.6476 - val_accuracy: 0.8497
Epoch 3/5
272/272 [==============================] - 55s 203ms/step - loss: 0.2042 - accuracy: 0.9491 - val_loss: 0.4138 - val_accuracy: 0.8905
Epoch 4/5
272/272 [==============================] - 58s 214ms/step - loss: 0.1291 - accuracy: 0.9688 - val_loss: 0.3651 - val_accuracy: 0.9138
Epoch 5/5
272/272 [==============================] - 58s 211ms/step - loss: 0.0889 - accuracy: 0.9791 - val_loss: 0.3558 - val_accuracy: 0.9045


In [550]:
print(history.history.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [551]:
print(f"Accuracy: {history.history['accuracy']}")

Accuracy: [0.5155320763587952, 0.8976407647132874, 0.94913649559021, 0.9688209295272827, 0.9791085720062256]


## 5. Evaluación del modelo

In [ ]:
# Evaluar el modelo entrenado utilizando el conjunto de prueba.
test_loss, test_accuracy = model.evaluate(X_test, y_test)

In [ ]:
# Mostrar cómo calcular métricas de evaluación, como Precisión, Recall y F1-Score para cada clase
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))